<a href="https://colab.research.google.com/github/defiglenn/learn-yearn-gitbook/blob/main/cmo2022-24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
from dateutil.parser import parse
from toolz import concat
from datetime import datetime, timezone

base_url = "https://safe-transaction-mainnet.safe.global"

transactions = []
address = "0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52"
next_page = f"{base_url}/api/v1/safes/{address}/multisig-transactions/"

while next_page:
    print(next_page)
    response = requests.get(next_page)

    # Check if the response is successful and contains JSON
    if response.status_code == 200 and 'json' in response.headers.get('Content-Type', ''):
        data = response.json()
        next_page = data["next"]
        transactions.extend(data["results"])
    else:
        # Handle cases where the response is not JSON (e.g., HTML)
        print(f"Non-JSON response from: {next_page}")
        # You might want to inspect the response.text here or break the loop
        break

    # Previous break condition, might be redundant now
    if (
        next_page
        == "https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/"
    ):
        break

def parse_transaction(tx):
    threshold_date = datetime(2022, 7, 1, tzinfo=timezone.utc)

    base = {
        "nonce": tx["nonce"],
        "submission_date": parse(tx["submissionDate"]),
        "execution_date": parse(tx["executionDate"]) if tx["executionDate"] else None,
    }
    confs = []
    for conf in tx["confirmations"]:
        execution_date = base["execution_date"]
        if execution_date and execution_date > threshold_date:
            confs.append(
                {
                    **base,
                    "owner": conf["owner"],
                    "confirmation_date": parse(conf["submissionDate"]),
            }
        )
    return confs


owner_map = {
    "0xf5D3dbda5F41A0E26D71B948e29522398e71cFaE" : "1",
    "0x7321ED86B0Eb914b789D6A4CcBDd3bB10f367153" : "2",
    "0x6F2A8Ee9452ba7d336b3fba03caC27f7818AeAD6" : "3",
    "0x2B888954421b424C5D3D9Ce9bB67c9bD47537d12" : "4",
    "0x0Cec743b8CE4Ef8802cAc0e5df18a180ed8402A7" : "5",
    "0x74630370197b4c4795bFEeF6645ee14F8cf8997D" : "6",
    "0x99BC02c239025E431D5741cC1DbA8CE77fc51CE3" : "7",
    "0x1496546f89fC1605880e556c9A1D6C5E2409fb0a" : "8",
    "0x7A1057E6e9093DA9C1D4C1D049609B6889fC4c67" : "9"
}

txs = list(concat(map(parse_transaction, transactions)))
df = pd.DataFrame(txs)
df["owner"] = df["owner"].apply(owner_map.get)
df["time_to_sign"] = df.confirmation_date - df.submission_date
df["time_to_execute"] = df.execution_date - df.submission_date

print(df.groupby("owner").describe()["time_to_sign"].sort_values("count", ascending=False)[["count", "mean", "50%", "max"]])
print(df.describe()["time_to_execute"])

https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=100
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=200
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=300
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=400
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=500
https://safe-transaction-mainnet.safe.global/api/v1/safes/0xFEB4acf3df3cDEA7399794D0869ef76A6EfAff52/multisig-transactions/?limit=100&offset=600
https:

In [ ]:
 count                       mean                        50%  \
owner
8       676  0 days 08:52:38.147070554     0 days 03:15:48.633694
6       595  0 days 12:58:44.445485115     0 days 05:02:35.094232
1       551  0 days 12:12:53.542667246     0 days 03:31:16.947480
9       512  0 days 14:00:37.218463160  0 days 03:33:44.622529500
5       510  0 days 12:45:43.016375298     0 days 02:58:08.374501
3       488  0 days 14:25:13.494660526  0 days 03:22:14.333387500
2       448  0 days 15:23:42.925990566     0 days 10:20:37.961621
4       290  0 days 19:21:32.014778834  0 days 07:42:57.005040500
7       177  0 days 16:14:27.104081751     0 days 04:24:59.015784

                           max
owner
8       4 days 22:11:25.547549
6       5 days 06:40:34.778323
1       7 days 09:02:18.239793
9       8 days 18:18:16.216909
5       7 days 08:56:33.363958
3       8 days 17:30:16.966884
2       9 days 02:13:48.977727
4       6 days 17:25:06.221971
7      10 days 02:47:02.966428
count                         4247
mean     1 days 20:01:07.901257352
std      1 days 14:45:30.938465403
min                0 days 00:00:00
25%         0 days 19:12:15.648273
50%         1 days 05:23:30.625591
75%         2 days 11:53:59.969874
max        10 days 03:12:14.172447
Name: time_to_execute, dtype: object